# Rekurentne neuronske mreže

Primer implementacije proste rekurentne neuronske mreže (RNN) *from scratch*, njeno treniranje na spisku imena dinosaurusa i primena za generisanje novog imena dinosaurusa.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

RNN su tip arhitekture koji je dobro prilagođen za obradu sekvencijalnih podataka kao što su tekst, audio, vremenske serije, itd. Ključni aspekt koji ih čini jedinstvenim je prisustvo rekurzivnih računarskih petlji koje obuhvataju susedne vremenske korake. Ovo ponavljanje omogućava modelima RNN da efikasno održavaju trajno interno stanje ili "memoriju" prethodnih ulaza koji bi mogli da informišu obradu podataka kasnije u sekvenci.

Vizuelno, tipična RNN struktura se sastoji od jedne računarske jedinice sa samopovezanim skrivenim stanjem, kroz koje informacije kruže kroz vremenske korake:

![Odmotavanje](images/unfold.png)

Kako podaci prolaze kroz RNN, aktivacije iz prethodnih vremenskih koraka teku kao ulazi na proračune mreže na trenutne podatke. Ovo omogućava modelu da dinamički inkorporira vremenski kontekst i istoriju sekvence. 

## Odmotavanje RNN

Umesto da posmatramo samo sažetu vizuelizaciju mreže, njeno odmotavanje oktriva ponavljajuću prirodu RNN izračunavanja, pretvarajući ga u tradicionalniniji usmereni aciklični računarski graf koji se sastoji od više vremenskih koraka. Tada možemo vizuelno videti kako se ulazna sekvenca obrađuje u svim vremenskim koracima, što pomaže razumevanju izračunavanja unapred i unazad kroz vreme kako bismo kasnije trenirali parametre.

![Forward](images/forward.png)

## *Forward pass*

*Forward pass* je mesto gde RNN obrađuje ulazne sekvence korak po korak kako bi računao predikcije.

1. **Računanje skrivenog stanja na osnovu ulaza**:

Za svaki vremenski korak $t$ u sekvenci, računamo skriveno stanje $h_t$ kao:  
$$ h_t = \tanh(U \cdot x_t + W \cdot h_{t-1} + b_h) $$  
gde:
- $U$ je matrica težina koja povezuje ulazni sloj sa skrivenim slojem
- $W$ je matrica težina koja povezuje prethodno skriveno stanje sa trenutnim skrivenim stanjem
- $b_h$ je *bias* vektor za skriveni sloj
- $tanh$ je aktivaciona funkcija koja uvodi nelinearnost. 

2. **Računanje izlaza na osnovu skrivenog stanja**:

Izlaz $y_t$ u svakom vremenskom koraku se računa kao:  
$$ y_t = \text{softmax}(V \cdot h_t + b_y) $$  
gde:  
- $V$ je matrica težina koja povezuje skriveni sloj sa izlaznim slojem
- $b_y$ je *bias* vektor za izlazni sloj
- $softmax$ normalizuje izlaze na distribuciju verovatnoće preko mogućih sledećih znakova.

3. **Čuvanje izlaza i skrivenih stanja**:

Čuvamo svako skriveno stanje $h_t$ i izlaz $y_t$ za upotrebu u *backward pass*-u.  

### Tok podataka

Na slici ispod vidimo primer toka podataka - pokušavamo da predvidimo sledeći znak na osnovu nekog niza znakova.

![Tok](images/flow.png)  

Ulazi su diskretni karakteri, koji se konvertuju u *one-hot* vektore. Skriveni sloj i izlazni sloj vrše proračune kao što je pomenuto u prethodnim formulama. Na osnovu izlaza *softmax* funkcije, možemo to pretvorditi nazad u prediktovanu reč. Svaki vremenski korak takođe ima prediktovani izlaz. $y_t$ u poslednjem vremenskom koraku je konačno predviđanje sledećeg karaktera uzlazne sekvence.

## *Backward pass*

Koristimo propagaciju u nazad kroz vreme (eng. *Backpropagation Through Time*).
Za odmotani računarski graf, slično onome kako sekvenca teče kroz mrežu sloj po sloj u *forward pass*-u, računamo gradijente u *backward pass*-u sloj po sloj, ali od poslednjeg vremenskog koraka ka početnom koraku, otuda i naziv "kroz vreme". *Loss* funkcija je unakrsna entropija (eng. *cross-entropy*), s tim što je proširujemo sa malom konstantom $1e-8$ koja će sprečiti greške u računanju.

![Backward pass](images/backward.png)

Računamo gradijente za svaku matricu težina kako bi unapredili model na osnovu grešaka predikcije:

1. **Greška izlaznog sloja**:

Greška na konačnom izlazu se računa kao:  
$$\delta_y = y_{\text{pred}} - y_{\text{true}}$$  
gde:
- $y_{\text{pred}}$ je prediktovana verovatnoća
- $y_{\text{true}}$ je *one-hot* enkodovana tačna vrednost.

Gradijenti za izlazni sloj:
$$\frac{\partial \text{Loss}}{\partial V} = \delta_y \cdot h_t^T$$
$$\frac{\partial \text{Loss}}{\partial b_y} = \delta_y$$

2. **Greška skrivenog sloja**:

Propagiramo grešku unazad kroz vreme da prilagodimo $W$, $U$ i $b_h$ za svaki prethodni vremenski korak. Za svaki korak $t$ računamo:  
$$\delta_h = (\delta_y \cdot V^T + \delta_h^{\text{next}}) \odot (1 - h_t^2)$$  
gde:  
- $\delta_h^{\text{next}}$ je greška iz sledećeg vremenskog koraka
- $(1 - h_t^2)$ je izvod $tanh$ funkcije.

Gradijenti za skriveni sloj:
$$\frac{\partial \text{Loss}}{\partial U} += \delta_h \cdot x_t^T$$
$$\frac{\partial \text{Loss}}{\partial W} += \delta_h \cdot h_{t-1}^T$$
$$\frac{\partial \text{Loss}}{\partial b_h} += \delta_h$$

3. **Ažuriranje težina**:

Nakon izračunavanja gradijenata u svim vremenskim koracima, ažuriramo svaki parametar na sledeći način:  
$$U = U - \eta \cdot \frac{\partial \text{Loss}}{\partial U}$$  
$$W = W - \eta \cdot \frac{\partial \text{Loss}}{\partial W}$$  
$$V = V - \eta \cdot \frac{\partial \text{Loss}}{\partial V}$$  
$$b_h = b_h - \eta \cdot \frac{\partial \text{Loss}}{\partial b_h}$$  
$$b_y = b_y - \eta \cdot \frac{\partial \text{Loss}}{\partial b_y}$$  
gde je $\eta$ stopa učenja (eng. *learning rate*).

## Implementacija

### Mreža

Implementacija klase koja predstavlja jednostavnu RNN sa metodama za *forward pass*, *backward pass*, treniranje i predikciju.

In [ ]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        # inicijalizacija matrica težina sa malim nasumičnim vrednostima
        self.U = np.random.uniform(-0.01, 0.01, (hidden_size, input_size))  # ulaz ka skrivenom
        self.W = np.random.uniform(-0.01, 0.01, (hidden_size, hidden_size))  # skriveni ka skrivenom
        self.V = np.random.uniform(-0.01, 0.01, (output_size, hidden_size))  # skriveni ka izlazu
        # inicijalizacija bias-a
        self.b_h = np.zeros((hidden_size, 1))  # bias za skriveni sloj
        self.b_y = np.zeros((output_size, 1))  # bias za izlazni sloj

    def forward(self, inputs):
        self.hidden_states = [np.zeros((self.W.shape[0], 1))]  # inicijalizacija skrivenog stanja
        self.outputs = [] # inicijalizacija liste za čuvanje predikcija u svakom vremenskom koraku

        # prolazak kroz svaki vremenski korak u ualznoj sekvenci
        for x_t in inputs:
            # novo skriveno stanje
            h_t = np.tanh(self.U @ x_t + self.W @ self.hidden_states[-1] + self.b_h)
            # novi izlaz
            y_t = self.softmax(self.V @ h_t + self.b_y)
            # cuvanje skrivenog stanja i izlaza u trenutnom koraku
            self.hidden_states.append(h_t)
            self.outputs.append(y_t)
        return self.outputs

    def backward(self, inputs, target, learning_rate = 0.01):
        # inicijalizacija gradijenata za matrice težina
        dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
        db_h, db_y = np.zeros_like(self.b_h), np.zeros_like(self.b_y)
        delta_h_next = np.zeros_like(self.hidden_states[0])

        # računanje greške izlaznog sloja u poslednjem vremenskom koraku
        delta_y = self.outputs[-1] - target
        dV += delta_y @ self.hidden_states[-1].T # gradijent izlazne matrice težina
        db_y += delta_y # gradijent za izlazni bias

        # BPTT 
        for t in reversed(range(len(inputs))):
            # greška skrivenog sloja
            delta_h = (self.V.T @ delta_y + delta_h_next) * (1 - self.hidden_states[t + 1] ** 2)
            # akumulacija gradijenata
            dU += delta_h @ inputs[t].T
            dW += delta_h @ self.hidden_states[t].T
            db_h += delta_h
            # ažuriranje za sledeći vremenski korak u BP petlji
            delta_h_next = self.W.T @ delta_h

        # ažuriranje teina i bias-ova sa akumuliranim gradijentima
        self.U -= learning_rate * dU
        self.W -= learning_rate * dW
        self.V -= learning_rate * dV
        self.b_h -= learning_rate * db_h
        self.b_y -= learning_rate * db_y

    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x, axis=0)

    def fit(self, inputs, targets, epochs = 1000, learning_rate = 0.01, verbose=0):
        # inicijalizacija liste za čuvanje loss-a za svaku epohu
        loss_history = []
        
        for epoch in range(epochs):
            epoch_loss = 0

            # procesiranje svake sekvence u trening skupu
            for i in range(len(inputs)):
                x_seq = inputs[i]
                y_true = targets[i]
                
                # forward pass za predikcije
                outputs = self.forward(x_seq)
                y_pred = outputs[-1] # uzimamo poslednju predikciju iz sekvence

                # računanje unakrsne entropije za sekvencu
                loss = -np.sum(y_true * np.log(y_pred + 1e-8))
                epoch_loss += loss # akumulacija loss-a za epohu

                # backward pass za ažuriranje težina
                self.backward(x_seq, y_true, learning_rate)
            
            loss_history.append(epoch_loss)
            if verbose:
                print(f"Epoch {epoch}, Loss: {epoch_loss:.4f}")
        
        # vizuelizacija loss-a
        plt.plot(loss_history)
        plt.title("Loss during training")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.show()

    def predict(self, start_sequence, char_to_index, index_to_char, max_length = 20):
        # inicijalizacija generisanog imena navedenom početnom sekvencom
        name = start_sequence
        # one-hot početne sekvence karaktera za ulaz u RNN
        input_seq = [self.one_hot_encode(char, len(char_to_index), char_to_index) for char in start_sequence]

        for _ in range(max_length - len(start_sequence)):
            # forward pass koristeći poslednji sequence_length karakter za predikciju
            output = self.forward(input_seq[-sequence_length:])[-1]
            # uzimanje indeksa karaktera sa najvećom verovatnoćom
            predicted_index = np.argmax(output)
            # konverzija indeksa u karakter
            predicted_char = index_to_char[predicted_index]
        
            # prekidanje generisanja ako se dođe do znaka za novu liniju
            if predicted_char == "\n":
                break

            # dodaj karakter na generisano ime
            name += predicted_char
            # dodaj one-hot enkoding prediktovanog karaktera za sledeći vremenski korak
            input_seq.append(self.one_hot_encode(predicted_char, len(char_to_index), char_to_index))
    
        return name

    @staticmethod
    def one_hot_encode(char, vocab_size, char_to_index):
        # generisanje one-hot vektora za dati karakter
        vector = np.zeros((vocab_size, 1))
        vector[char_to_index[char]] = 1
        return vector

### Trening podaci

In [ ]:
with open("data/dinos.txt", "r") as f:
    names = f.read().splitlines()

print(len(names))

### Priprema vokabulara

In [ ]:
vocab = sorted(set("".join(names)))
print(len(vocab))
print(vocab)
char_to_index = {char: idx for idx, char in enumerate(vocab)}
index_to_char = {idx: char for char, idx in char_to_index.items()}

### Priprema parova ulaz-izlaz

In [ ]:
sequence_length = 10
inputs, targets = [], []
for name in names:
    encoded_name = [RNN.one_hot_encode(char, len(char_to_index), char_to_index) for char in name]
    for i in range(len(encoded_name) - sequence_length):
        inputs.append(encoded_name[i:i + sequence_length]) # ulazna sekvenca
        targets.append(encoded_name[i + sequence_length]) # sledeći karakter

print(len(inputs))
print(len(targets))

### Inicijalizacija modela

In [ ]:
input_size = len(char_to_index)
hidden_size = 50
output_size = len(char_to_index)
rnn = RNN(input_size, hidden_size, output_size)

### Trening

In [ ]:
rnn.fit(inputs, targets, epochs=20, learning_rate=0.01, verbose=1)

### Predikcija

In [ ]:
print("Generated Dinosaur Name:", rnn.predict("A", char_to_index, index_to_char))